In [ ]:
CREATE TABLE compounds (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    inchi TEXT UNIQUE,
    formula TEXT,
    molecular_weight REAL,
    date_added TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    added_by TEXT
);


In [ ]:
CREATE TABLE experiments (
    id SERIAL PRIMARY KEY,
    compound_id INTEGER REFERENCES compounds(id) ON DELETE CASCADE,
    description TEXT,
    date DATE,
    performed_by TEXT
);


In [ ]:
CREATE TABLE analytical_data (
    id SERIAL PRIMARY KEY,
    experiment_id INTEGER REFERENCES experiments(id) ON DELETE CASCADE,
    technique TEXT, -- e.g., NMR, MS, IR
    file_path TEXT, -- or BLOB if storing data in the DB
    notes TEXT
);
